In [ ]:
#default_exp radardsp

# radardsp

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

import numpy as np
from mmwave import dsp

In [ ]:
#export

def cfar_processing(radar_cube):
    range_az = np.abs(radar_cube)
    heatmap_log = np.log2(range_az)

    first_pass, _ = np.apply_along_axis(func1d=dsp.cago_,
                                        axis=0,
                                        arr=heatmap_log,
                                        l_bound=1.5,
                                        guard_len=4,
                                        noise_len=16)

    # --- cfar in range direction
    second_pass, noise_floor = np.apply_along_axis(func1d=dsp.caso_,
                                                   axis=0,
                                                   arr=heatmap_log.T,
                                                   l_bound=3,
                                                   guard_len=4,
                                                   noise_len=16)

    # --- classify peaks and caclulate snrs
    SKIP_SIZE = 4
    noise_floor = noise_floor.T
    first_pass = (heatmap_log > first_pass)
    second_pass = (heatmap_log > second_pass.T)
    peaks = (first_pass & second_pass)
    peaks[:SKIP_SIZE, :] = 0
    peaks[-SKIP_SIZE:, :] = 0
    peaks[:, :SKIP_SIZE] = 0
    peaks[:, -SKIP_SIZE:] = 0

    peaks = peaks.astype('float32')
    return peaks
